In [ ]:
# Aligns a score volume with an annotation volume

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

import logging

from registration_utilities import *

In [3]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
colors[labels_index['BackG']] = 1.

In [4]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'

In [5]:
volume1 = bp.unpack_ndarray_file(os.path.join(volume_dir, 'volume_MD589_annotation.bp'))

In [6]:
def parallel_where(l):
    w = np.where(volume1 == l)
    return np.array([w[1].astype(np.int16), w[0].astype(np.int16), w[2].astype(np.int16)]).T

t = time.time()

atlas_nzs = Parallel(n_jobs=16)(delayed(parallel_where)(l) for l in range(1, n_labels))

print time.time() - t, 'seconds'

# atlas_xmin, atlas_ymin, atlas_zmin = np.min([np.min(atlas_nzs[l-1], axis=1) for l in range(1, n_labels)], axis=0)
# atlas_xmax, atlas_ymax, atlas_zmax = np.max([np.max(atlas_nzs[l-1], axis=1) for l in range(1, n_labels)], axis=0)
# print atlas_xmin, atlas_xmax, atlas_ymin, atlas_ymax, atlas_zmin, atlas_zmax

# atlas_centroid = np.array([.5*atlas_xmin+.5*atlas_xmax, .5*atlas_ymin+.5*atlas_ymax, .5*atlas_zmin+.5*atlas_zmax])
# print atlas_centroid

# atlas_cx, atlas_cy, atlas_cz = atlas_centroid

atlas_ydim, atlas_xdim, atlas_zdim = volume1.shape
atlas_centroid = (.5*atlas_xdim, .5*atlas_ydim, .5*atlas_zdim)
print atlas_centroid

1.20207500458 seconds
(405.0, 202.5, 267.0)


In [7]:
downsample_factor = 16

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

In [8]:
atlasAlignOptLogs_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_atlasAlignOptLogs'
if not os.path.exists(atlasAlignOptLogs_dir):
    os.makedirs(atlasAlignOptLogs_dir)

In [9]:
atlasAlignParams_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_atlasAlignParams'
if not os.path.exists(atlasAlignParams_dir):
    os.makedirs(atlasAlignParams_dir)

In [10]:
annotationsViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_annotaionsPojectedViz'

In [11]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [13]:
volume2_allLabels = None

In [16]:
ds = []
for l in range(1, n_labels):
    ds.append(np.array(atlas_nzs[l-1]) - atlas_centroid)

In [17]:
def transform_points(T, pts=None, c=None, pts_centered=None, c_prime=0):
    '''
    T: 1x12 vector
    c: center of volume 1
    c_prime: center of volume 2
    pts: nx3
    '''
    
    if pts_centered is None:
        pts_centered = pts - c
    
    Tm = np.reshape(T, (3,4))
    t = Tm[:, 3]
    A = Tm[:, :3]
        
    pts_prime = np.dot(A, pts_centered.T) + (t + c_prime)[:,None]
        
    return pts_prime.T

In [34]:
def compute_score_and_gradient(T):
    global ds
    
    score = 0
    dMdA = np.zeros((12,))
    
    for l in range(1, n_labels):
#         t1 = time.time()
    
        pts_prime = transform_points(T, pts_centered=ds[l-1], c_prime=test_centroid)
    
        xs_prime = pts_prime[:,0]
        ys_prime = pts_prime[:,1]
        zs_prime = pts_prime[:,2]

        valid = (xs_prime >= 0) & (ys_prime >= 0) & (zs_prime >= 0) & \
                (xs_prime < test_xdim) & (ys_prime < test_ydim) & (zs_prime < test_zdim)
                   
        if np.count_nonzero(valid) > 0:

            xs_prime_valid = xs_prime[valid].astype(np.int16)
            ys_prime_valid = ys_prime[valid].astype(np.int16)
            zs_prime_valid = zs_prime[valid].astype(np.int16)
            
            voxel_probs_valid = volume2_allLabels[l-1][ys_prime_valid, xs_prime_valid, zs_prime_valid] / 1e4

            score += voxel_probs_valid.sum()
            
            Sx = dSdxyz[l-1][0][ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Sy = dSdxyz[l-1][1][ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Sz = dSdxyz[l-1][2][ys_prime_valid, xs_prime_valid, zs_prime_valid]
            
            dxs, dys, dzs = ds[l-1][:, valid]

#             dMdA += np.c_[Sx*dxs, Sx*dys, Sx*dzs, Sx, 
#                           Sy*dxs, Sy*dys, Sy*dzs, Sy,
#                           Sz*dxs, Sz*dys, Sz*dzs, Sz].sum(axis=0)
            
            q = np.c_[Sx*dxs, Sx*dys, Sx*dzs, Sx, 
                          Sy*dxs, Sy*dys, Sy*dzs, Sy,
                          Sz*dxs, Sz*dys, Sz*dzs, Sz]        
            
            dMdA += q.sum(axis=0)
            
            del voxel_probs_valid, q, Sx, Sy, Sz, dxs, dys, dzs, xs_prime_valid, ys_prime_valid, zs_prime_valid
        
#         sys.stderr.write('########### %s: %f seconds\n' % (labels[l], time.time() - t1))
        
        del valid, xs_prime, ys_prime, zs_prime, pts_prime
        
    return score, dMdA

In [35]:
def compute_score(T):
    
    score = 0
    for l in range(1, n_labels):

        pts_prime = transform_points(T, pts_centered=ds[l-1], c_prime=test_centroid)
    
        xs_prime = pts_prime[:,0].astype(np.int16)
        ys_prime = pts_prime[:,1].astype(np.int16)
        zs_prime = pts_prime[:,2].astype(np.int16)
        
        valid = (xs_prime >= 0) & (ys_prime >= 0) & (zs_prime >= 0) & \
            (xs_prime < test_xdim) & (ys_prime < test_ydim) & (zs_prime < test_zdim)
        voxel_probs_valid = volume2_allLabels[l-1][ys_prime[valid], xs_prime[valid], zs_prime[valid]] / 1e4

        score += voxel_probs_valid.sum()
                
    del voxel_probs_valid, valid, xs_prime, ys_prime, zs_prime, pts_prime
                
    return score

def compute_score_gradient(T):


    dMdA = np.zeros((12,))

    for l in range(1, n_labels):
        
        pts_prime = transform_points(T, pts_centered=ds[l-1], c_prime=test_centroid)

        xs_prime = pts_prime[:,0].astype(np.int16)
        ys_prime = pts_prime[:,1].astype(np.int16)
        zs_prime = pts_prime[:,2].astype(np.int16)

        valid = (xs_prime >= 0) & (ys_prime >= 0) & (zs_prime >= 0) & \
            (xs_prime < test_xdim) & (ys_prime < test_ydim) & (zs_prime < test_zdim)
            
        if np.count_nonzero(valid) > 0:
            
            xs_prime_valid = xs_prime[valid]
            ys_prime_valid = ys_prime[valid]
            zs_prime_valid = zs_prime[valid]
            
            Sx = dSdxyz[l-1][0][ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Sy = dSdxyz[l-1][1][ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Sz = dSdxyz[l-1][2][ys_prime_valid, xs_prime_valid, zs_prime_valid]
               
            dxs, dys, dzs = ds[l-1][:, valid]
            dMdA += np.c_[Sx*dxs, Sx*dys, Sx*dzs, Sx, 
                          Sy*dxs, Sy*dys, Sy*dzs, Sy,
                          Sz*dxs, Sz*dys, Sz*dzs, Sz].sum(axis=0)
            
    return dMdA

In [ ]:
stack = 'MD598'

In [24]:
# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD598']:

################# LOAD TEST VOLUME ######################

section_bs_begin, section_bs_end = section_range_lookup[stack]
print section_bs_begin, section_bs_end

(volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
np.loadtxt(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_limits.txt' % {'stack': stack}), dtype=np.int)

map_z_to_section = {}
for s in range(section_bs_begin, section_bs_end+1):
    for z in range(int(z_xy_ratio_downsampled*s) - volume_zmin, int(z_xy_ratio_downsampled*(s+1)) - volume_zmin + 1):
        map_z_to_section[z] = s

global volume2_allLabels
volume2_allLabels = []

for l in labels[1:]:

    print l

    volume2 = bp.unpack_ndarray_file(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_%(label)s.bp' % \
                                                  {'stack': stack, 'label': l}))

    volume2_cropped = volume2[volume_ymin:volume_ymax+1, volume_xmin:volume_xmax+1]
    # copy is important, because then you can delete the large array

    volume2_allLabels.append(volume2_cropped.copy())

    del volume2, volume2_cropped

test_ydim, test_xdim, test_zdim = volume2_allLabels[0].shape
test_centroid = (.5*test_xdim, .5*test_ydim, .5*test_ydim)
test_cx, test_cy, test_cz = test_centroid

print test_xdim, test_ydim, test_zdim
print test_centroid

95 354
5N
7n
7N
12N
Pn
VLL
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
897 459 410
(448.5, 229.5, 229.5)


In [28]:
handler = logging.FileHandler(atlasAlignOptLogs_dir + '/%(stack)s_atlasAlignOpt.log' % {'stack': stack})
handler.setLevel(logging.INFO)
logger.addHandler(handler)

################# GRID SEARCH ######################

#     grid_search_iteration_number = 10
grid_search_iteration_number = 1

params_best_upToNow = (0, 0, 0)
score_best_upToNow = 0

for iteration in range(grid_search_iteration_number):

    logger.info('grid search iteration %d', iteration)

    init_tx, init_ty, init_tz  = params_best_upToNow

    n = int(1000*np.exp(-iteration/3.))

    sigma_tx = 300*np.exp(-iteration/3.)
    sigma_ty = 300*np.exp(-iteration/3.)
    sigma_tz = 100*np.exp(-iteration/3.)

    tx_grid = init_tx + sigma_tx * (2 * np.random.random(n) - 1)
    ty_grid = init_ty + sigma_ty * (2 * np.random.random(n) - 1)
    tz_grid = init_tz + sigma_tz * (2 * np.random.random(n) - 1)

    samples = np.c_[tx_grid, ty_grid, tz_grid]

    import time
    t = time.time()

    scores = Parallel(n_jobs=16)(delayed(compute_score)(np.array([1, 0, 0, tx, 0, 1, 0, ty, 0, 0, 1, tz]))
                                 for tx, ty, tz in samples)

#     scores = [compute_score([1, 0, 0, tx, 0, 1, 0, ty, 0, 0, 1, tz]) for tx, ty, tz in samples]

    print time.time() - t, 'seconds'

    score_best = np.max(scores)

    tx_best, ty_best, tz_best = samples[np.argmax(scores)]

    if score_best > score_best_upToNow:
        logger.info('%f %f', score_best_upToNow, score_best)

        score_best_upToNow = score_best
        params_best_upToNow = tx_best, ty_best, tz_best

        logger.info('%f %f %f', tx_best, ty_best, tz_best)
        logger.info('\n')

INFO:__main__:grid search iteration 0
INFO:__main__:0.000000 75.142848
INFO:__main__:-61.942285 55.148047 -65.577031
INFO:__main__:



46.6715641022 seconds


In [31]:
dSdxyz = np.empty((n_labels-1, 3) + volume2_allLabels[0].shape, dtype=np.float16) 

# if memory is not limited, using float32 is faster, because the output of np.gradient is of type float32
# time for storing output: float16 4s (due to dtype conversion overhead), float32 1s

# using float16 avoids memory issues that make gradient computation utterly slow, 30s vs. 4s

################# COMPUTE GRADIENTS ######################

# dSdxyz = {}
# DO NOT use python list because python will use contiguous memory for it
# http://stackoverflow.com/questions/12274060/does-python-use-linked-lists-for-lists-why-is-inserting-slow  

t1 = time.time()

for l in range(1, n_labels):

    t = time.time()
    
    gy, gx, gz = np.gradient(volume2_allLabels[l-1], 3, 3, 3) # 3.3 second, much faster than loading
    # if memory is limited, this will be very slow
    
    sys.stderr.write('gradient %s: %f seconds\n' % (labels[l], time.time() - t))
    
    t = time.time()
    
    dSdxyz[l-1, 0] = gx
    dSdxyz[l-1, 1] = gy
    dSdxyz[l-1, 2] = gz
    
#     dSdxyz[labels[l]] = np.array([gx, gy, gz]) # use np.array is better; using python list also causes contiguous memory overhead
    
#     del gx, gy, gz # does not make a difference
    
    sys.stderr.write('store %s: %f seconds\n' % (labels[l], time.time() - t))
    
sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

gradient 5N: 3.194637 seconds
store 5N: 3.999422 seconds
gradient 7n: 3.143157 seconds
store 7n: 3.780772 seconds
gradient 7N: 3.164200 seconds
store 7N: 3.866401 seconds
gradient 12N: 3.147282 seconds
store 12N: 3.800223 seconds
gradient Pn: 3.370479 seconds
store Pn: 3.852262 seconds
gradient VLL: 3.527252 seconds
store VLL: 4.146684 seconds
gradient 6N: 3.594103 seconds
store 6N: 3.810074 seconds
gradient Amb: 3.693601 seconds
store Amb: 3.828036 seconds
gradient R: 6.101754 seconds
store R: 4.068227 seconds
gradient Tz: 20.304137 seconds
store Tz: 3.865987 seconds
gradient RtTg: 24.929295 seconds
store RtTg: 4.155099 seconds
gradient LRt: 31.895824 seconds
store LRt: 3.938405 seconds
gradient LC: 28.409347 seconds
store LC: 8.762701 seconds
gradient AP: 23.406426 seconds
store AP: 5.801922 seconds
gradient sp5: 22.389653 seconds
store sp5: 6.839631 seconds
overall: 252.800252 seconds


In [36]:
################# GRADIENT DESCENT ######################

lr1, lr2 = (10., 1e-1)
# lr1, lr2 = (1., 1e-3)

# auto_corr = .95

max_iter_num = 100
fudge_factor = 1e-6 #for numerical stability
dMdA_historical = np.zeros((12,))

tx_best, ty_best, tz_best = params_best_upToNow
T_best = np.r_[1,0,0, tx_best, 0,1,0, ty_best, 0,0,1, tz_best]

lr = np.array([lr2, lr2, lr2, lr1, lr2, lr2, lr2, lr1, lr2, lr2, lr2, lr1])

score_best = 0

scores = []

for iteration in range(max_iter_num):

    logger.info('iteration %d\n', iteration)

    t = time.time()
    s, dMdA = compute_score_and_gradient(T_best)
    sys.stderr.write('compute_score_and_gradient: %f seconds\n' % (time.time() - t)) #~ 2s/iteration

    dMdA_historical += dMdA**2
#     dMdA_historical = auto_corr * dMdA_historical + (1-auto_corr) * dMdA**2

    dMdA_adjusted = dMdA / (fudge_factor + np.sqrt(dMdA_historical))

    T_best += lr*dMdA_adjusted

#         logger.info('A: ' + ' '.join(['%f']*12) % tuple(A_best))
#         logger.info('dMdA adjusted: ' + ' '.join(['%f']*12) % tuple(dMdA_adjusted))

    logger.info('score: %f', s)
    scores.append(s)

#     logger.info('\n')

    history_len = 50
    if iteration > 200:
        if np.abs(np.mean(scores[iteration-history_len:iteration]) - \
                  np.mean(scores[iteration-2*history_len:iteration-history_len])) < 1e-1:
            break

    if s > score_best:
#             logger.info('Current best')
        best_gradient_descent_params = T_best
        score_best = s

#     np.save(atlasAlignOptLogs_dir + '/%(stack)s_scoreEvolutions.npy' % {'stack':stack}, scores)

#     del dSdxyz

################# PROJECT ATLAS TO IMAGE ######################


#     Tm = np.reshape(best_gradient_descent_params, (3,4))
#     tx_best, ty_best, tz_best  = Tm[:, 3]
#     Amat_best = Tm[:, :3]

#     atlas_nzs_projected_to_test = [(np.dot(Amat_best, vs - atlas_centroid[:, np.newaxis]) + \
#                                                 np.asarray([tx_best + test_cx, 
#                                                             ty_best + test_cy, 
#                                                             tz_best + test_cz])[:,np.newaxis]).astype(np.int)
#                                     for vs in atlas_nzs]

#     print np.min(atlas_nzs_projected_to_test[0], axis=1)
#     print np.max(atlas_nzs_projected_to_test[0], axis=1)

#     test_volume_atlas_projected = np.zeros_like(volume2_allLabels[0], np.int)

#     for l in range(1, n_labels):

#         test_xs, test_ys, test_zs = atlas_nzs_projected_to_test[l-1].astype(np.int)

#         valid = (test_xs >= 0) & (test_ys >= 0) & (test_zs >= 0) & \
#             (test_xs < test_xdim) & (test_ys < test_ydim) & (test_zs < test_zdim)

#         atlas_xs, atlas_ys, atlas_zs = atlas_nzs[l-1]

#         test_volume_atlas_projected[test_ys[valid], test_xs[valid], test_zs[valid]] = \
#         volume1[atlas_ys[valid], atlas_xs[valid], atlas_zs[valid]]


#     del atlas_nzs_projected_to_test

#     bp.pack_ndarray_file(test_volume_atlas_projected, 
#                          volume_dir + '/%(stack)s_volume_atlasProjected.bp'%{'stack':stack})

#     with open(os.path.join(atlasAlignParams_dir, '%(stack)s_3dAlignParams.txt' % {'stack':stack}), 'w') as f:
#         f.writelines(' '.join(['%f']*len(params_best_upToNow)) % tuple(params_best_upToNow) + '\n')
#         f.writelines(' '.join(['%f']*len(best_gradient_descent_params)) % tuple(best_gradient_descent_params) + '\n')
#         f.writelines(' '.join(['%f']*len(lr)) % tuple(lr) + '\n')
#         f.writelines('%d' % iteration + '\n')

#     annotationsViz_dir = create_if_not_exists(annotationsViz_rootdir + '/' + stack)

#     dm = DataManager(stack=stack)

#     for z in range(0, test_zdim, 10):

#         t = time.time()

#         print z

#         dm.set_slice(map_z_to_section[z])
#         dm._load_image(versions=['rgb-jpg'])
#         viz = dm.image_rgb_jpg[::downsample_factor, ::downsample_factor][volume_ymin:volume_ymax+1, 
#                                                                          volume_xmin:volume_xmax+1].copy()

#         projected_cnts = find_contour_points(test_volume_atlas_projected[...,z])

#         for label_ind, cnts in projected_cnts.iteritems():
#             for cnt in cnts:
#                 cv2.polylines(viz, [cnt.astype(np.int)], True, tuple((colors[label_ind]*255).astype(np.int)), 2)

#         cv2.imwrite(annotationsViz_dir + '/%(stack)s_%(sec)04d_annotationsProjectedViz_z%(z)04d.jpg' % \
#                     {'stack': stack, 'sec': map_z_to_section[z], 'z': z}, 
#                     img_as_ubyte(viz[..., [2,1,0]]))

#         del viz

#         sys.stderr.write('visualize: %f seconds\n' % (time.time() - t)) #~ 2s/iteration


#     del test_volume_atlas_projected

#     logger.removeHandler(handler)


INFO:__main__:iteration 0

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:32: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 3 but corresponding boolean dimension is 447348


MemoryError: 